In [25]:
import models.MLP.model as MLP
import models.CNN.model as CNN
import importlib
import numpy as np
from tqdm import tqdm
import torch

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset
from tqdm.auto import tqdm
import itertools
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import models.learning as L

transform_MNIST = transforms.Compose([
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.view(-1))
        ])
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=False, transform=transform_MNIST)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=False, transform=transform_MNIST)

train_dataloader = DataLoader(train_dataset, batch_size=1)
test_dataloader = DataLoader(test_dataset, batch_size=1)

In [11]:
import importlib

In [52]:
importlib.reload(MLP)
importlib.reload(L)

<module 'models.learning' from '/Users/zhaoyizhe/Documents/GitHub/Hebbian_Topdown/models/learning.py'>

In [8]:
device = torch.device('cpu')

In [55]:
model = MLP.MLPBaseline_Model(64, 100, 0.005, 0.01, 1e-3, 0.99, 10, device, MLP.ClassifierLearning.Contrastive, MLP.Learning.FullyOrthogonal, MLP.WeightScale.WeightDecay)
model = MLP.TDBaseline_Experiment(1, model, train_dataloader, 'FashionMNIST', 10, device)

  0%|          | 0/60000 [00:00<?, ?it/s]

KeyboardInterrupt: 